In [1]:
import gymnasium as gym
from stable_baselines3 import SAC, TD3, A2C
import os
import argparse
from stable_baselines3.common.noise import NormalActionNoise
import numpy as np

2025-04-20 19:54:46.003881: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:


# Create base directories
model_base_dir = "models"
log_base_dir = "logs"
os.makedirs(model_base_dir, exist_ok=True)
os.makedirs(log_base_dir, exist_ok=True)

name = "SAC_"

# Find next available index
existing_dirs = [d for d in os.listdir(log_base_dir) if d.startswith(name) and d[4:].isdigit()]
next_index = max([int(d[4:]) for d in existing_dirs], default=-1) + 1

# Create log and model directories with same name
log_dir = os.path.join(log_base_dir, f"{name}{next_index}")
model_dir = os.path.join(model_base_dir, f"{name}{next_index}")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

def train(model, timesteps):
    iters = 0
    while True:
        iters += 1
        model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
        # Save to models/SAC_X/
        model.save(f"{model_dir}/model_{model.num_timesteps}")
    return model
# def train(model, timesteps, ):

    

#     iters = 0
#     while True:
#         iters += 1
#         model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
#         model.save(f"{model_dir}{name}{next_index}{model.num_timesteps - model.num_timesteps % 1000}")
#     return model





In [10]:
env = gym.make("Humanoid-v5", render_mode=None)


In [11]:
action_noise = NormalActionNoise(mean=np.zeros(env.action_space.shape), 
                                 sigma=0.15 * np.ones(env.action_space.shape))
# model = SAC('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    device="cuda",
    tensorboard_log=log_dir,
    learning_rate=0.0004,        # Default, but can be reduced for more exploration
    batch_size=512,              # Increase batch size for stability
    tau=0.004,                   # Lower tau to slow down target updates
    gamma=0.98,                  # Slightly reduce gamma to value short-term rewards more
    ent_coef=0.06,               # Encourage more entropy (more randomness)
    target_update_interval=2,     # Update targets slightly less frequently
    action_noise=action_noise,     # Inject additional exploration noise
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
# model = SAC.load("models/SAC_736000", verbose=1, device='cuda', tensorboard_log=log_dir)
# model.set_env(env)

In [13]:
model = train(model, 50)

Logging to logs/SAC_3/SAC_0
Logging to logs/SAC_3/SAC_0
Logging to logs/SAC_3/SAC_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 26.5     |
|    ep_rew_mean     | 123      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 50       |
|    time_elapsed    | 0        |
|    total_timesteps | 106      |
| train/             |          |
|    actor_loss      | -2.14    |
|    critic_loss     | 13.3     |
|    ent_coef        | 0.06     |
|    learning_rate   | 0.0004   |
|    n_updates       | 5        |
---------------------------------
Logging to logs/SAC_3/SAC_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.6     |
|    ep_rew_mean     | 105      |
| time/              |          |
|    episodes        | 8        |
|    fps             | 41       |
|    time_elapsed    | 0        |
|    total_timesteps | 181      |
| train/             |          |
|   

KeyboardInterrupt: 

In [12]:
model.save("models/SAC_1")

NameError: name 'model' is not defined